In [ ]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv("processed_dulieuxettuyendaihoc.csv")

# 1. Xác định và phân loại dữ liệu
danh_muc = ['GT', 'DT', 'KV', 'KT']  # Dữ liệu định tính
so_luong = [col for col in df.columns if col not in danh_muc]  # Dữ liệu định lượng

print("\n--- Dữ liệu định tính ---")
print(danh_muc)
print("\n--- Dữ liệu định lượng ---")
print(so_luong)

# 2. Định nghĩa thang đo phù hợp
measurement_scales = {col: 'Nominal' for col in danh_muc}
for col in so_luong:
    measurement_scales[col] = 'Ratio'

print("\n--- Thang đo dữ liệu ---")
print(measurement_scales)

# 3. In 10 dòng đầu và cuối
print("\n--- 10 dòng đầu tiên ---")
print(df.head(10))
print("\n--- 10 dòng cuối cùng ---")
print(df.tail(10))

# 4. Xử lý dữ liệu thiếu cho cột DT
print("\n--- Thống kê dữ liệu thiếu của DT ---")
print(df['DT'].isnull().sum())
df['DT'] = df['DT'].fillna(0)
print("\n--- Sau khi thay thế dữ liệu thiếu của DT ---")
print(df['DT'].unique())

# 5. Thống kê dữ liệu thiếu và thay thế bằng Mean cho T1
print("\n--- Thống kê dữ liệu thiếu của T1 ---")
print(df['T1'].isnull().sum())
df['T1'] = df['T1'].fillna(df['T1'].mean())
print("\n--- Sau khi thay thế dữ liệu thiếu của T1 bằng Mean ---")
print(df['T1'].describe())

# 6. Xử lý dữ liệu thiếu cho tất cả các cột điểm còn lại
cols_to_fill = ['T1', 'L1', 'H1', 'S1', 'V1', 'X1', 'D1', 'N1',
                'T2', 'L2', 'H2', 'S2', 'V2', 'X2', 'D2', 'N2',
                'T6', 'L6', 'H6', 'S6', 'V6', 'X6', 'D6', 'N6',
                'DH1', 'DH2', 'DH3']
df[cols_to_fill] = df[cols_to_fill].apply(lambda x: x.fillna(x.mean()))
print("\n--- Kiểm tra dữ liệu thiếu sau khi xử lý ---")
print(df.isnull().sum())

# 7. Tạo biến TBM1, TBM2, TBM3
def tinh_TBM(row, t, l, h, s, v, x, d, n):
    return (row[t]*2 + row[l] + row[h] + row[s] + row[v]*2 + row[x] + row[d] + row[n]) / 10

df['TBM1'] = df.apply(lambda row: tinh_TBM(row, 'T1', 'L1', 'H1', 'S1', 'V1', 'X1', 'D1', 'N1'), axis=1)
df['TBM2'] = df.apply(lambda row: tinh_TBM(row, 'T2', 'L2', 'H2', 'S2', 'V2', 'X2', 'D2', 'N2'), axis=1)
df['TBM3'] = df.apply(lambda row: tinh_TBM(row, 'T6', 'L6', 'H6', 'S6', 'V6', 'X6', 'D6', 'N6'), axis=1)
print("\n--- Kiểm tra TBM ---")
print(df[['TBM1', 'TBM2', 'TBM3']].head())

# 8. Tạo các biến xếp loại
def xep_loai(diem):
    if diem < 5.0:
        return 'Y'
    elif diem < 6.5:
        return 'TB'
    elif diem < 8.0:
        return 'K'
    elif diem < 9.0:
        return 'G'
    else:
        return 'XS'

df['XL1'] = df['TBM1'].apply(xep_loai)
df['XL2'] = df['TBM2'].apply(xep_loai)
df['XL3'] = df['TBM3'].apply(xep_loai)
print("\n--- Kiểm tra xếp loại ---")
print(df[['XL1', 'XL2', 'XL3']].head())

# 9. Chuyển đổi sang thang điểm 4
def min_max_scaling(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val) * 4

df['US_TBM1'] = df['TBM1'].apply(lambda x: min_max_scaling(x, df['TBM1'].min(), df['TBM1'].max()))
df['US_TBM2'] = df['TBM2'].apply(lambda x: min_max_scaling(x, df['TBM2'].min(), df['TBM2'].max()))
df['US_TBM3'] = df['TBM3'].apply(lambda x: min_max_scaling(x, df['TBM3'].min(), df['TBM3'].max()))
print("\n--- Kiểm tra thang điểm 4 ---")
print(df[['US_TBM1', 'US_TBM2', 'US_TBM3']].head())

# 10. Xác định KQXT
def xet_tuyen(row):
    if row['KT'] in ['A', 'A1']:
        return 1 if ((row['DH1'] * 2 + row['DH2'] + row['DH3']) / 4) >= 5.0 else 0
    elif row['KT'] == 'B':
        return 1 if ((row['DH1'] + row['DH2'] * 2 + row['DH3']) / 4) >= 5.0 else 0
    else:
        return 1 if ((row['DH1'] + row['DH2'] + row['DH3']) / 3) >= 5.0 else 0

df['KQXT'] = df.apply(xet_tuyen, axis=1)
print("\n--- Kiểm tra kết quả xét tuyển ---")
print(df[['KT', 'KQXT']].head())
# 11. Lưu dữ liệu xuống file
output_path = "D:/processed_dulieuxettuyendaihoc.csv"
df.to_csv(output_path, index=False)
print(f"Dữ liệu đã được lưu tại {output_path}")





--- Dữ liệu định tính ---
['GT', 'DT', 'KV', 'KT']

--- Dữ liệu định lượng ---
['STT', 'T1', 'L1', 'H1', 'S1', 'V1', 'X1', 'D1', 'N1', 'T2', 'L2', 'H2', 'S2', 'V2', 'X2', 'D2', 'N2', 'T3', 'L3', 'H3', 'S3', 'V3', 'X3', 'D3', 'N3', 'T4', 'L4', 'H4', 'S4', 'V4', 'X4', 'D4', 'N4', 'T5', 'L5', 'H5', 'S5', 'V5', 'X5', 'D5', 'N5', 'T6', 'L6', 'H6', 'S6', 'V6', 'X6', 'D6', 'N6', 'DH1', 'DH2', 'DH3', 'TBM1', 'TBM2', 'TBM3', 'XL1', 'XL2', 'XL3', 'US_TBM1', 'US_TBM2', 'US_TBM3', 'KQXT']

--- Thang đo dữ liệu ---
{'GT': 'Nominal', 'DT': 'Nominal', 'KV': 'Nominal', 'KT': 'Nominal', 'STT': 'Ratio', 'T1': 'Ratio', 'L1': 'Ratio', 'H1': 'Ratio', 'S1': 'Ratio', 'V1': 'Ratio', 'X1': 'Ratio', 'D1': 'Ratio', 'N1': 'Ratio', 'T2': 'Ratio', 'L2': 'Ratio', 'H2': 'Ratio', 'S2': 'Ratio', 'V2': 'Ratio', 'X2': 'Ratio', 'D2': 'Ratio', 'N2': 'Ratio', 'T3': 'Ratio', 'L3': 'Ratio', 'H3': 'Ratio', 'S3': 'Ratio', 'V3': 'Ratio', 'X3': 'Ratio', 'D3': 'Ratio', 'N3': 'Ratio', 'T4': 'Ratio', 'L4': 'Ratio', 'H4': 'Ratio', '